## (실습) 마스크 영상 2
* 마스크 영상에 의해 지정된 일부 영역만 복사하기

In [2]:
# 비행기 사진에서 마스킹 영역만 가져와서 필드 영상에 복사하기 (덮어씌우기)
import cv2
import numpy as np

def mask_copyTo():
    src = cv2.imread('airplane.bmp', cv2.IMREAD_COLOR)
    mask = cv2.imread('mask_plane.bmp', cv2.IMREAD_GRAYSCALE)
    dst = cv2.imread('field.bmp', cv2.IMREAD_COLOR)
    
    if src is None or mask is None or dst is None:
        print('Image load failed!')
        return
    
    dst[mask > 0] = src[mask > 0] # 마스크에서 흰색 영역만 복사됨
    
    cv2.imshow('src', src)
    cv2.imshow('dst', dst)
    cv2.imshow('mask', mask)
    cv2.imwrite('result.png', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
mask_copyTo()

## 연산 시간 측정
* OpenCV 라이브러리를 이용하면 운영체제에 상관 없이 연산 시간 측정 가능
* tm = **cv2.TickMeter()** 클래스
* tm.start()
* tm.stop()
* **tm.getTimeMilli()** : ms 단위로 시간 측정 (계산 시간 가져오기)

## (실습) 연산 시간 측정
* For문을 활용해 영상을 직접 한 픽셀씩 반전시키고, 이때 소요되는 시간을 측정하여 출력

In [8]:
# 1. for문 사용시 연산 시간 측정

def time_inverse():
    src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)
    
    dst = np.empty(src.shape, dtype=src.dtype) # 초기화 없이 배열 생성
    
    tm = cv2.TickMeter()
    tm.start()
    
    for y in range(src.shape[0]): # row
        for x in range(src.shape[1]): # col
            dst[y, x] = 255 - src[y, x]
            
    tm.stop()
    print('Image inverse implementation took %4.3f ms.'% tm.getTimeMilli())

time_inverse()

Image inverse implementation took 586.288 ms.


In [9]:
# 2. ~ 연산자 (numpy 벡터화 연산) 사용시 연산 시간 측정 ==> 훨씬 빠르다!

def time_inverse2():
    src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)
    
    dst = np.empty(src.shape, dtype=src.dtype) # 초기화 없이 배열 생성
    
    tm = cv2.TickMeter()
    tm.start()
    
    dst = ~src # 벡터화 연산 (numpy 연산)
            
    tm.stop()
    print('Image inverse implementation took %4.3f ms.'% tm.getTimeMilli())

time_inverse2()

Image inverse implementation took 0.387 ms.


In [5]:
# cf. np.empty : 특정 값으로 초기화해주지 않아 빠르지만
# 임의의 값들이 들어가있어서 추후 값을 직접 재할당 해주어야 한다!

test = np.empty((3, 3), dtype=np.uint8)
print(test)

[[176  22 135]
 [184 249 127]
 [  0   0  96]]


## 유용한 OpenCV, NumPy 함수 사용법
* **np.sum()** : 행렬의 전체 원소 합
* **np.mean()** : 행렬의 전체 원소 평균 / 마스크 연산 지원 (특정 영역의 평균)
    * 둘 다 4채널 이하의 행렬에 대해서만 동작 (RGBA)

* **cv2.minMaxLoc()** : 행렬의 **최솟값, 최댓값**을 찾는 함수. 해당 값의 **좌표**도 함께 알수있음 (min, max, location)
    * 파이썬에서는 **tuple** 형태로 return
    * **grayscale만** 가능

* **cv2.normalize()** : 행렬의 norm값을 정규화하거나 원소 값 범위를 특정 범위로 정규화할 때 사용 (**0 ~ 1 사이 값**으로) / **"norm_type"** 인자에 따라 동작 결정
     
    * norm_type = **NORM_IMF** : L-infinity norm (**최대 절댓값**을 반환)
        * ex) x = [-6, 4, 2] ==> norm : 6
          
    * norm_type = **NORM_L1** : L1 norm (Manhattan Distance, **두 점 사이 직각 거리**. 즉, **각각의 차의 절댓값의 합**)
        * ex) (0, 0) ~ (3, 4) ==> norm : |3| + |4| = 7
          
    * norm_type = **NORM_L2** : L2 norm (Euclidean Distance, **각각의 차의 제곱 합의 루트**) 
        * ex) (0, 0) ~ (3, 4) ==> norm : 5
      > 만약 norm_type = **NORM_MINMAX** 이면, src 행렬의 최솟값이 **alpha**, 최댓값이 **beta**가 되도록 원소값 크기 조절 (대부분 이렇게 사용)

* **round()** : 파이썬 **내장** 함수. 실수를 정수로 변환 (**반올림**)

## (실습) np.sum, np.mean

In [10]:
def useful_func():
    img = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)
    
    if img is None :
        print('Image load failed!')
        return
    
    sum_img = np.sum(img)
    mean_img = np.mean(img, dtype = np.int32)
    print('sum:', sum_img)
    print('mean:', mean_img)

useful_func()

sum: 28890183
mean: 110


In [12]:
# cf. np.mean()과 마스크 연산 ==> 특정 마스크 영역의 원소 평균 구하기

def mean_mask():
    img = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread('mask_smile.bmp', cv2.IMREAD_GRAYSCALE)
    
    mean_mask_img = np.mean(img[mask > 0], dtype = np.int32) # 마스크 흰색 영역 평균
    print('mean_mask:', mean_mask_img)

mean_mask()

mean_mask: 109


## (실습) cv2.minMaxLoc()
* 단, img는 **grayscale만** 가능!

In [19]:
img = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE) # Grayscale만 됨!!
minVal, maxVal, minPos, maxPos = cv2.minMaxLoc(img)
print(cv2.minMaxLoc(img))
print('minVal is', minVal, 'at', minPos)
print('maxVal is', maxVal, 'at', maxPos)

(22.0, 239.0, (508, 71), (117, 272))
minVal is 22.0 at (508, 71)
maxVal is 239.0 at (117, 272)


## (실습) cv2.normalize()
* -1에서 1 사이의 실수로 구성된 1 x 5 행렬을 0부터 255 사이의 정수 행렬로 변환하기

In [23]:
# 최솟값 : 0, 최댓값 : 255 로 normalize
src = np.array([[-1, -0.5, 0, 0.5, 1]], dtype = np.float32)
dst = cv2.normalize(src, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U) # uint8(8bit)행렬
print('src:', src)
print('dst:', dst)

src: [[-1.  -0.5  0.   0.5  1. ]]
dst: [[  0  64 128 191 255]]


## (실습) round()
정확히 **0.5**인 경우에는 **짝/홀수**에 따라서 다름!

In [25]:
print(round(2.5)) # 짝수.5 ==> 내림
print(round(2.51))
print(round(3.499))
print(round(3.5)) # 홀수.5 ==> 올림

2
3
3
4


<hr>

# Chap 5. 영상의 밝기와 명암비 조절
* 영상의 밝기 조절 (cv2.add(), cv2.subtract())
* 영상의 명암비 조절 (cv2.multiply())
* 히스토그램 분석

# 영상의 밝기 조절
* cv2.cvtColor(srt, dst, 컬러변환코드)
* saturate() : 포화 연산 (범위를 벗어나는 값 가질 경우 자료형의 최대 or 최대값으로 설정)
    * if) uchar 자료형 ==> 0 ~ 255
* 실제로 영상의 밝기 조절 시 **포화연산** 고려해야 함!
> **cv2.add()** & **cv2.subtract()** 함수 활용

In [27]:
# cv2.add(), cv2.subtract() ==> 자동으로 포화연산 수행

import numpy as np
import cv2

def brightness1():
    src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)
    dst1 = cv2.add(src, 100) # 덧셈 연산
    dst2 = cv2.subtract(src, 100) # 뺄셈 연산
    
    cv2.imshow('src', src)
    cv2.imshow('dst_add', dst1)
    cv2.imshow('dst_subtract', dst2)
    cv2.waitKey()
    cv2.destroyAllWindows()

brightness1()

In [28]:
# 영상의 밝기 조절 직접 +, -로 구현하기 ==> 포화연산 적용 안되어서 찢어지는 부분 발생

def brightness2():
    src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)
    dst = np.empty(src.shape, src.dtype)
    
    for y in range(src.shape[0]):
        for x in range(src.shape[1]):
            dst[y, x] = src[y, x] + 100 # 255 넘는 값(overflow) : 0부터로 처리
    
    cv2.imshow('src', src)
    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()

brightness2()

In [29]:
# 이중 for문을 수정하여 정상적인 밝기 조절 구현 ==> 3항연산자로 0~255 넘는 값 처리

def saturated(value):
    if value > 255:
        value = 255
    elif value < 0:
        value = 0
    
    return value

def brightness3():
    src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)
    dst = np.empty(src.shape, src.dtype)
    
    for y in range(src.shape[0]):
        for x in range(src.shape[1]):
            dst[y, x] = saturated(src[y, x] + 100) # 모든 픽셀마다 포화연산 함수로
    
    cv2.imshow('src', src)
    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()

brightness3()

## 트랙바를 이용한 영상의 밝기 조절
* 매번 소스 코드 수정과 빌드 필요 없이, add() 함수와 트랙바를 통해 곧바로 결과 확인 가능
* cv2.createTrackbar(트랙바 이름, 트랙바 생성할 창 이름, 시작 위치, 최대 위치, 콜백함수명)

In [31]:
# 원본 영상 + 0~100 사이 값 조절
def brightness4():
    src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)
    
    # 트랙바 콜백함수
    def update(pos): 
        dst = cv2.add(src, pos) # src + pos(트랙바 값)
        cv2.imshow('dst', dst)
    
    cv2.namedWindow('dst')
    cv2.createTrackbar('Brightness', 'dst', 0, 100, update) # update : 콜백함수
    update(0) # 처음에 보여줄 이미지 (디폴트 pos = 0)
    
    cv2.waitKey()
    cv2.destroyAllWindows()

brightness4()

# 영상의 명암비 조절

## 기본적인 명암비 조절
* 명암비 == **명암 대비** == 콘트라스트 (**contrast**)
* 밝은 영역과 어두운 영역 사이에 드러나는 밝기 차이의 강도
* "**명암비가 낮다**" == 전반적으로 어둡거나 전반적으로 밝은 픽셀로만 구성된 경우
    * 객체 간 구분이 잘 되지 않아서 전반적으로 **흐림**
* "**명암비가 높다**" == 밝고 어두운 영역이 **골고루** 섞여 있다
    * 사물의 구분이 잘 되며 **선명함**

## 기본적인 명암비 조절 방법
* **cv2.multiply()** 함수 이용 (실전에서는 잘 안씀)
* dst = saturated(s * src)에서,
    * **s = 0.5** : 0~255 -> 0~128 이므로 밝기 차이(기울기)가 줄어듦 (**명암비 작아짐**)
    * **s = 2** : 0~255 -> 원본의 128 밝기부분이 255가 되어 큰 밝기로 변함 (**명암비 커짐**)

In [33]:
# cv2.multiply() ==> 모든 픽셀값에 2배하여 명암대비 크게 하기

def contrast1():
    src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)
    
    s = 2.0
    dst = cv2.multiply(src, s)
    
    cv2.imshow('src', src)
    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()

contrast1() 

# 전체적으로 포화되어 흰색 영역이 너무 많아서 윤곽 구분 더 어려워짐
# 실전에서는 곱셈을 이용해 명암비 조절 방식 잘 안씀!